---

# Chapter 15: Service-Oriented Architecture (SOA) and Microservices

## Opening Context

As software systems grow beyond a certain size, the limitations of a single monolithic application become apparent. Deployment becomes slow, scaling is inefficient, and teams step on each other's toes. Service‑Oriented Architecture (SOA) emerged to address these challenges by decomposing applications into discrete, reusable services that communicate over a network. In recent years, **microservices** have refined SOA principles, advocating for small, independently deployable services.

This chapter explores three critical aspects of service‑oriented design:

1. **Microservices vs. Monoliths** – A decision framework to help you choose the right architectural style.
2. **The Service Layer Pattern** – How to encapsulate business logic behind a clean service interface, a fundamental building block in both monolithic and distributed systems.
3. **Backend for Frontend (BFF)** – A pattern that tailors backend services to the specific needs of different clients, improving efficiency and developer experience.

Understanding these concepts will help you design systems that are scalable, maintainable, and aligned with your organisation’s goals.

---

## 15.1 Microservices vs. Monoliths: Decision Frameworks

### The Monolithic Architecture

A **monolith** is a single deployable unit that contains all the functionality of an application. Typically, it has a modular internal structure (e.g., layers), but the entire application is built, tested, and deployed together.

```typescript
// Simplified monolithic e-commerce application structure
src/
├── controllers/       // HTTP request handling
├── services/          // Business logic (OrderService, ProductService, etc.)
├── repositories/      // Data access
├── models/            // Domain entities
└── app.ts             // Entry point that ties everything together
```

**Pros**:
- **Simple development** – One codebase, easy to run locally.
- **Simple deployment** – Deploy one artifact.
- **Simple testing** – End‑to‑end tests can cover everything.
- **Low latency** – All communication is in‑process.

**Cons**:
- **Scaling inefficiency** – You must scale the entire application even if only one component is under load.
- **Team coordination** – Many teams working on the same codebase lead to merge conflicts and slow release cycles.
- **Technology lock‑in** – Difficult to adopt new technologies incrementally.
- **Reliability** – A bug in any part can bring down the whole system.

### Microservices Architecture

**Microservices** are a style of SOA where each service is small, focuses on a single business capability, and can be deployed independently. Services communicate over lightweight protocols (HTTP/REST, gRPC, message queues).

```typescript
// Microservice structure for Order service (separate repository)
order-service/
├── src/
│   ├── controllers/    // HTTP endpoints for order operations
│   ├── domain/         // Order entity, value objects
│   ├── repositories/   // Database access for orders
│   ├── clients/        // Clients to call other services (product, payment)
│   └── app.ts
└── package.json
```

**Pros**:
- **Independent scaling** – Scale only the services that need it.
- **Team autonomy** – Teams can own services end‑to‑end.
- **Technology diversity** – Each service can use the best language/database for its task.
- **Resilience** – Failure in one service is isolated (if designed well).

**Cons**:
- **Distributed complexity** – Network latency, service discovery, retries, circuit breakers.
- **Data consistency** – Transactions across services require sagas or eventual consistency.
- **Testing complexity** – Integration tests require multiple services.
- **Operational overhead** – Monitoring, logging, and deployment become more complex.

### Decision Framework: Monolith vs. Microservices

Choosing between monolith and microservices is not binary; many successful systems start as monoliths and gradually decompose. Use the following framework to guide your decision.

| Factor | Favour Monolith | Favour Microservices |
|--------|------------------|----------------------|
| **Team Size** | Small team (<10) | Multiple teams (>5) |
| **Domain Complexity** | Simple CRUD | Complex domain with clear bounded contexts |
| **Scalability Requirements** | Moderate, uniform load | High, uneven load across components |
| **Release Velocity** | Slow, coordinated releases | Frequent, independent releases |
| **Technology Evolution** | Stable tech stack | Desire to experiment with new technologies |
| **Organisational Structure** | Co‑located, one team | Distributed teams, each owning a product area |

#### The "Monolith First" Strategy

For most new projects, start with a well‑structured monolith. Ensure it has **clean module boundaries** (using packages/modules) so that if you later need to split, you can extract modules into services. This approach avoids the upfront complexity of distributed systems while keeping the option open.

#### Decision Flowchart

```
Start: New Project?
    |
    v
Is team size > 5 and likely to grow?
    / \
  Yes   No
   |     |
   v     v
Domain has clear subdomains?  --> Likely start with monolith
    / \
  Yes   No
   |     |
   v     v
Consider microservices   --> Start modular monolith
    |                           |
    v                           v
Build with bounded contexts     Design for future extraction
```

---

## 15.2 The Service Layer Pattern

### Intent
*Define a boundary that encapsulates the application's business logic, providing a coarse‑grained API to clients (such as UI controllers or other services). The service layer coordinates transactions, enforces business rules, and delegates to domain objects or repositories.*

### The Problem

In many applications, business logic is scattered across controllers, views, or even stored procedures. This makes the system hard to understand, test, and maintain. Moreover, when you later decide to expose the same functionality via different interfaces (e.g., REST API and message queue), you end up duplicating code.

### The Solution: Service Layer

The Service Layer pattern centralises business logic into a set of classes that represent the use cases of the application. Each service method corresponds to a user‑level operation (e.g., `placeOrder`, `cancelOrder`). The service layer sits between the presentation layer (controllers) and the data access layer (repositories).

```typescript
// domain/order.ts (Domain entity)
export class Order {
  constructor(
    public readonly id: string,
    public readonly customerId: string,
    public items: OrderItem[],
    public status: OrderStatus
  ) {}

  addItem(productId: string, quantity: number, price: number): void {
    this.items.push({ productId, quantity, price });
    // Possibly recalculate total, raise domain events, etc.
  }

  cancel(): void {
    if (this.status === OrderStatus.SHIPPED) {
      throw new Error('Cannot cancel shipped order');
    }
    this.status = OrderStatus.CANCELLED;
  }
}
```

```typescript
// repositories/order.repository.ts
import { Order } from '../domain/order';

export interface OrderRepository {
  save(order: Order): Promise<void>;
  findById(id: string): Promise<Order | null>;
}
```

```typescript
// services/order.service.ts (Service Layer)
import { Order } from '../domain/order';
import { OrderRepository } from '../repositories/order.repository';
import { PaymentService } from './payment.service'; // another service or client

export class OrderService {
  constructor(
    private orderRepository: OrderRepository,
    private paymentService: PaymentService
  ) {}

  async placeOrder(customerId: string, items: OrderItem[]): Promise<string> {
    // Create order entity
    const order = new Order(generateId(), customerId, items, OrderStatus.PENDING);
    
    // Business rule: validate items
    if (items.length === 0) throw new Error('Order must have items');
    
    // Possibly call payment service to authorize payment
    await this.paymentService.authorize(order.total);
    
    // Save order
    await this.orderRepository.save(order);
    
    // Return order id
    return order.id;
  }

  async cancelOrder(orderId: string): Promise<void> {
    const order = await this.orderRepository.findById(orderId);
    if (!order) throw new Error('Order not found');
    
    order.cancel(); // domain logic
    await this.orderRepository.save(order);
  }
}
```

**Explanation**:
- The service layer uses domain entities and repositories.
- It orchestrates business processes (e.g., calling payment service) and enforces high‑level rules (e.g., order must have items).
- It does **not** contain HTTP or persistence logic; those are delegated to repositories and external service clients.
- The service layer can be used by multiple adapters: a REST controller, a GraphQL resolver, a message queue listener, etc.

### Service Layer in a Microservices Context

In a microservices architecture, each service typically has its own service layer. The service layer may call other services via HTTP clients (or messaging). For example, the `OrderService` above calls a `PaymentService` which is a separate microservice. To maintain loose coupling, we define an interface for the payment client:

```typescript
// clients/payment.client.ts (in Order service)
export interface PaymentClient {
  authorize(amount: number): Promise<void>;
}

// Implementation using HTTP
export class HttpPaymentClient implements PaymentClient {
  constructor(private baseUrl: string) {}
  
  async authorize(amount: number): Promise<void> {
    const response = await fetch(`${this.baseUrl}/authorize`, {
      method: 'POST',
      body: JSON.stringify({ amount })
    });
    if (!response.ok) throw new Error('Payment failed');
  }
}
```

Then inject this client into the `OrderService`. This keeps the service layer decoupled from the transport details.

### Testing the Service Layer

One of the key benefits of the service layer is testability.

```typescript
// test/order.service.test.ts
import { OrderService } from '../services/order.service';
import { InMemoryOrderRepository } from '../repositories/in-memory-order.repository';
import { MockPaymentClient } from '../clients/mock-payment.client';

describe('OrderService', () => {
  it('should place an order successfully', async () => {
    const repo = new InMemoryOrderRepository();
    const paymentClient = new MockPaymentClient();
    const service = new OrderService(repo, paymentClient);
    
    const orderId = await service.placeOrder('cust-1', [
      { productId: 'p1', quantity: 2, price: 10 }
    ]);
    
    const savedOrder = await repo.findById(orderId);
    expect(savedOrder).toBeDefined();
    expect(savedOrder?.status).toBe('PENDING');
  });
});
```

**Explanation**:
- We use an in‑memory repository and a mock payment client to test the service in isolation.
- No database or HTTP calls are involved; tests run fast and reliably.

### Relationship to Other Patterns

The Service Layer pattern is often combined with:
- **Domain Model** – The service layer uses rich domain objects.
- **Repository** – For data access.
- **Unit of Work** – To manage transactions across repositories.
- **DTOs (Data Transfer Objects)** – To decouple the service layer from the presentation layer.

---

## 15.3 Backend for Frontend (BFF)

### Intent
*Create separate backend services for each frontend application (e.g., mobile, web, desktop) to optimise the API for that specific client, reducing overhead and simplifying frontend code.*

### The Problem

In a traditional multi‑client architecture, a single backend API serves all clients. This leads to several issues:

- **Over‑fetching** – Mobile clients may receive data they don't need, wasting bandwidth.
- **Under‑fetching** – A client may need to make multiple round trips to assemble the required data.
- **Different interaction patterns** – Mobile apps may require different authentication flows or data formats than web apps.
- **Backend changes** – A change to support one client may inadvertently break another.

### The Solution: BFF

The **Backend for Frontend** pattern introduces a dedicated backend for each frontend. Each BFF is tailored to the specific needs of its client. The BFF may aggregate data from multiple downstream services, transform data into a client‑friendly format, and implement client‑specific logic (e.g., mobile‑optimised caching).

```
[Web Client] <--> [Web BFF] <---> [Product Service]
                                     [Order Service]
[Mobile Client] <--> [Mobile BFF] <-- [User Service]
                                     [Recommendation Service]
```

In this diagram, the Web BFF and Mobile BFF are separate services. They may both call the same underlying microservices, but they compose responses differently.

### Example: BFF for Mobile vs. Web

Assume we have three downstream services:
- `user-service` – provides user profile data.
- `order-service` – provides order history.
- `product-service` – provides product details.

#### Mobile BFF

Mobile screens often need a dashboard combining user info, recent orders, and recommended products. The mobile BFF can fetch all this in a single request.

```typescript
// mobile-bff/controllers/dashboard.controller.ts
import { Request, Response } from 'express';
import { UserClient } from '../clients/user.client';
import { OrderClient } from '../clients/order.client';
import { ProductClient } from '../clients/product.client';

export class DashboardController {
  constructor(
    private userClient: UserClient,
    private orderClient: OrderClient,
    private productClient: ProductClient
  ) {}

  async getDashboard(req: Request, res: Response): Promise<void> {
    const userId = req.user.id; // from auth middleware
    
    // Fetch data in parallel
    const [user, orders, recommendations] = await Promise.all([
      this.userClient.getProfile(userId),
      this.orderClient.getRecentOrders(userId, 5),
      this.productClient.getRecommendations(userId, 10)
    ]);
    
    // Transform to mobile‑optimised response (e.g., smaller payload, flattened structure)
    res.json({
      user: { name: user.name, avatar: user.avatarThumbnail },
      recentOrders: orders.map(o => ({ id: o.id, date: o.date, total: o.total })),
      recommendations: recommendations.map(p => ({ id: p.id, name: p.name, price: p.price }))
    });
  }
}
```

#### Web BFF

The web client might need more detailed information for a product page, including reviews and specifications. The web BFF can combine product data with reviews.

```typescript
// web-bff/controllers/product.controller.ts
import { Request, Response } from 'express';
import { ProductClient } from '../clients/product.client';
import { ReviewClient } from '../clients/review.client';

export class ProductController {
  constructor(
    private productClient: ProductClient,
    private reviewClient: ReviewClient
  ) {}

  async getProduct(req: Request, res: Response): Promise<void> {
    const productId = req.params.id;
    
    const [product, reviews] = await Promise.all([
      this.productClient.getProduct(productId),
      this.reviewClient.getReviews(productId)
    ]);
    
    // Enrich with additional web‑specific data (e.g., related articles)
    res.json({
      ...product,
      reviews,
      relatedArticles: [] // could come from another service
    });
  }
}
```

### Benefits of BFF

- **Optimised Payloads** – Each client receives exactly what it needs.
- **Reduced Round Trips** – The BFF can aggregate multiple downstream calls into one.
- **Client‑Specific Logic** – Authentication, caching, and error handling can be tailored.
- **Decoupling** – Frontend teams can own their BFF, evolving it independently of other clients.

### Drawbacks

- **Duplication** – Some logic may be duplicated across BFFs (e.g., authentication, logging).
- **Operational Overhead** – More services to deploy and monitor.
- **Latency** – The BFF adds an extra network hop, though this is often mitigated by co‑location.

### When to Use BFF

- You have multiple, significantly different clients (e.g., mobile, web, third‑party API).
- Clients have distinct data requirements or interaction patterns.
- You want to give frontend teams autonomy over their backend dependencies.

### BFF vs. API Gateway

An **API Gateway** is a single entry point that routes requests to appropriate services, often handling cross‑cutting concerns like authentication, rate limiting, and logging. A **BFF** is a specialised form of API Gateway tailored to a specific client. In practice, you might have an API Gateway that then routes to BFFs, or each BFF could act as its own gateway.

---

## Chapter Summary

This chapter covered three important patterns for structuring service‑oriented systems:

1. **Microservices vs. Monoliths** – We examined the trade‑offs and provided a decision framework to help choose the right architecture. The key is to consider team size, domain complexity, and scalability needs. Starting with a well‑structured monolith is often the safest path, allowing gradual migration to microservices when justified.

2. **The Service Layer Pattern** – This pattern centralises business logic into use‑case classes, decoupling it from presentation and data access. It improves testability, reusability, and maintainability. In a microservices context, the service layer becomes the core of each service, orchestrating calls to other services via clients.

3. **Backend for Frontend (BFF)** – BFF addresses the challenge of serving multiple diverse clients by creating dedicated backends for each. This optimises data transfer, reduces round trips, and gives frontend teams autonomy. While it adds operational complexity, it can greatly improve client performance and developer productivity.

**Key Insight**: The principles of separation of concerns and encapsulation apply at every level—from classes to services. Whether you're designing a monolithic service layer or a constellation of microservices, the goal is the same: create clear boundaries that allow independent evolution and testing.

---

## Next Chapter Preview

**Chapter 16: Resilience and Fault Tolerance (Circuit Breaker, Retry, Bulkhead)**

In distributed systems, failures are inevitable. Chapter 16 will explore patterns that make your services resilient to failures: **Circuit Breaker** to prevent cascading failures, **Retry with Exponential Backoff** to handle transient errors, and **Bulkhead** to isolate failures and protect critical resources. These patterns are essential for building robust, production‑ready microservices.

